*How many qualitative and quantitative variables are there? What are appropiate visuals for quantitative vs qualitative data? What are appropiate measures for correlations when dealing with qualitative and quantitative variables?

What is the correlation between the variables and the price? Why do you think some variables are more correlated than others?

How are the variables themselves correlated to each other? Can you find groups of variables that are correlated together?*

In [ ]:
#Import clean data file

In [ ]:
#Check data types and define qualitative and quantitative variables

In [ ]:
# Corr between price and other variables

In [ ]:
#Multicollinearity (between variables, exl price)